In [2]:
from ultralytics import YOLO
import os
import cv2
import cvzone
import math
import numpy as np

In [3]:
os.chdir('../../../../')

In [4]:
from sort.sort import Sort

In [6]:
model = YOLO("./AI/computer vision/yolov11/yolo11n.pt")
print(model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [7]:
video_path = "./AI/computer vision/Projects/objects_counter/road.mp4"
cap = cv2.VideoCapture(video_path)
mask = cv2.imread('./AI/computer vision/Projects/objects_counter/mask.png')
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

IDs = [1,2,3,5,7]
#          x1, x2,  y
limits = [5, 400, 297]
totalCount = []

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.resize(frame ,(960,540))

    frameMask = cv2.bitwise_and(frame, frame, mask=mask)

    results = model(frameMask, stream=True)
    detection = np.empty((0,5)) # 0 rows 5 columns

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0] # for cv2
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),1)

            w, h = x2 - x1, y2 - y1 # for cvZone

            id = int(box.cls[0])
            conf = math.ceil(box.conf[0]*100) / 100

            if id in IDs and conf > 0.5:
                # cvzone.cornerRect(frame, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
                # # print(conf)
                # cvzone.putTextRect(frame, f"{conf} {model.names[id]}", (max(0, x1), max(35, y1)), scale=1, thickness=1)

                currArray = np.array([x1, y1, x2, y2, conf])
                detection = np.vstack((detection, currArray)) # add one row to the array
    
    results = tracker.update(detection)
    cv2.line(frame, (limits[0], limits[2]), (limits[1], limits[2]), (0, 0, 255), 5)

    for result in results:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(frame, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(frame, f' {int(id)}', (max(0, x1), max(35, y1)),
                        scale=2, thickness=3, offset=10)
        cx ,cy = (x1 + w // 2), (y1 + h // 2)
        cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        if limits[2] - 15 < cy < limits[2] + 15:
            if id not in totalCount:
                totalCount.append(id)
            
            cv2.line(frame, (limits[0], limits[2]), (limits[1], limits[2]), (0, 255, 0), 5)
        
        cv2.putText(frame,str(len(totalCount)),(20,60),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)


    cv2.imshow("frame", frame)
    # cv2.imshow("frameMask", frameMask)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 1 truck, 203.5ms
Speed: 4.5ms preprocess, 203.5ms inference, 8.7ms postprocess per image at shape (1, 3, 384, 640)
[        198         135         279         220           2]
[          0         291          62         355           1]


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ahmedcr7/.local/lib/python3.11/site-packages/cv2/qt/plugins"



0: 384x640 2 cars, 1 truck, 241.0ms
Speed: 9.3ms preprocess, 241.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
[     196.01      134.99      277.99      221.01           2]
[     -1.163      297.36      54.164      359.64           1]

0: 384x640 2 cars, 1 truck, 167.2ms
Speed: 5.8ms preprocess, 167.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[     193.12      135.72      275.06      221.28           2]
[    -2.4656      305.22      44.584      364.66           1]

0: 384x640 2 cars, 1 truck, 154.8ms
Speed: 6.4ms preprocess, 154.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
[     190.26      137.49      272.72      223.05           2]
[    -3.1786      312.41      35.963      368.81           1]

0: 384x640 2 cars, 1 truck, 127.8ms
Speed: 17.8ms preprocess, 127.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[     187.47      138.18      272.04      225.57           2]
[     -3.461      319.05 